**NHÓM C2: Performance Testing (Locust)**

# Code phụ: Selenium trên ứng dụng Gmail

Mục đích: Minh hoạ việc gửi Gmail tự động bằng Selenium với demo riêng biệt trước khi tích hợp vào Locust

**Môi trường thực thi:**

- MacOS Sonoma 14.0
- CPU Apple M1
- Python 3.10.10
- Trình duyệt Microsoft Edge Version 118.0.2088.76 (Official build) (arm64)

## Cài đặt Selenium

In [1]:
!pip install selenium

## Import thư viện

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import datetime
import os

## Hàm tạo tiêu đề và nội dung email

In [3]:
def create_email():
    # Lấy ngày giờ hiện tại
    current_datetime = datetime.datetime.now()

    # Format ngày giờ theo dạng "hh:mm:ss dd/mm/yyyy"
    formatted_datetime = current_datetime.strftime("%H:%M:%S %d/%m/%Y")


    email = {
        "title" : f"Locust Testing at {formatted_datetime}",
        "content" : f"This is a test (executed at {formatted_datetime})."
    }

    return email

create_email_example = create_email()
print(f"Example:")
print(f'''-> Title  : "{create_email_example['title']}"''')
print(f'''-> Content: "{create_email_example['content']}"''')

Example:
-> Title  : "Locust Testing at 17:08:52 20/11/2023"
-> Content: "This is a test (executed at 17:08:52 20/11/2023)."


## Hàm đổi màu phần tử trên trang web

In [4]:
def change_element_color(browser, element):
    browser.execute_script(
        "arguments[0].style.backgroundColor = 'lightgreen';", 
        element
    )

## Hàm đọc file văn bản

In [5]:
def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            file_contents = file.read()
        return file_contents
    except FileNotFoundError:
        return f"File not found: {file_path}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

## Địa chỉ email gửi và email nhận

In [6]:
# Tạo file lưu password (để không trình chiếu password khi demo)
# !touch "password.pass" # Lệnh này không gây ảnh hưởng nếu file đã tồn tại

source_email = "pqvuong.dev.testing@gmail.com"
source_password = read_file("password.pass")
dest_email = "20120406@student.hcmus.edu.vn"

## Microsoft Edge Remote Debugging Port (tránh bị nhận diện phần mềm automation)

In [7]:
browser_instance_data_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKI/Kiem thu phan mem/Seminar/Locust-Seminar/Advanced Demo/browser_instance_data/test_instance"
port_to_run_browser = 39999

command = f'''open -na "Microsoft Edge.app" --args
            --remote-debugging-port={port_to_run_browser}
            --user-data-dir="{browser_instance_data_dir}"'''

os.system(command.replace("\n", " "))

0

## Kết nối Selenium đến trình duyệt Edge ở port được mở

In [8]:
options = webdriver.EdgeOptions()
options.add_experimental_option("debuggerAddress", f"localhost:{port_to_run_browser}")
# options.add_argument("headless") # Chạy Selenium ở chế độ headless (không hiển thị giao diện browser)

browser = webdriver.Edge(options=options)
browser.maximize_window() # Phóng to cửa sổ trình duyệt

## Đi đến trang đăng nhập Gmail

In [9]:
browser.get('https://mail.google.com/mail/u/0/')

## Nhập email và bấm next để đến trang nhập mật khẩu

In [10]:
def enter_email(browser, email):
    username_input_field = browser.find_element(
        By.ID, 'identifierId'
    )
    change_element_color(browser, username_input_field)
    username_input_field.send_keys(email)

    next_button_on_username_page = browser.find_element(
        By.XPATH,
        '//button[.//span[contains(text(),"Next") or contains(text(),"Tiếp theo")]]'
    )
    change_element_color(browser, next_button_on_username_page)
    next_button_on_username_page.click()

enter_email(browser, source_email)

## Nhập mật khẩu và nhấn next để đăng nhập

In [11]:
def enter_password(browser, password):
    password_input_field = browser.find_element(
            By.XPATH, 
            '//input[contains(@aria-label,"Enter your password") or contains(@aria-label,"Nhập mật khẩu của bạn")]'
        )
    change_element_color(browser, password_input_field)
    password_input_field.send_keys(password)

    next_button_on_password_page = browser.find_element(
        By.XPATH,
        '//button[.//span[contains(text(),"Next") or contains(text(),"Tiếp theo")]]'
    )
    change_element_color(browser, next_button_on_password_page)
    next_button_on_password_page.click()

enter_password(browser, source_password)

## Nhấn nút soạn thư

In [12]:
def start_a_new_email(browser):
    new_email_button = browser.find_element(
        By.XPATH,
        "//*[text()='Soạn thư']"
    )
    change_element_color(browser, new_email_button)
    new_email_button.click()

start_a_new_email(browser)

## Nhập người nhận, tiêu đề và nội dung email

In [13]:
def enter_email_details(browser, dest_email):
    email = create_email()

    # Nhập người nhận
    dest_email_field = browser.find_element(
        By.XPATH, 
        "//input[@role='combobox']"
    )
    change_element_color(browser, dest_email_field)
    dest_email_field.send_keys(dest_email)

    # Nhập tiêu đề
    email_title_field = browser.find_element(
        By.XPATH, 
        "//input[@name='subjectbox']"
    )
    change_element_color(browser, email_title_field)
    email_title_field.send_keys(email["title"])

    # Nhập nội dung email
    email_content_field = browser.find_element(
        By.XPATH,
        "//div[@aria-label='Nội dung thư']"
    )
    change_element_color(browser, email_content_field)
    email_content_field.send_keys(email["content"])

enter_email_details(browser, dest_email)

## Nhấn nút gửi

In [14]:
def send_email(browser):
    send_email_button = browser.find_element(
        By.XPATH, 
        "//div[text()='Gửi']"
    )
    change_element_color(browser, send_email_button)
    send_email_button.click()

send_email(browser)